# k-medias

In [1]:
import pandas as pd

In [2]:
Data = pd.read_csv('House_Price_Regression.csv')

In [3]:
Data = Data.loc[:, ['latitude', 'longitude', 'price', 'size_in_m_2']]

Data.head()

,latitude,longitude,price,size_in_m_2
0,25.113208,55.138932,2700000,100.242337
1,25.106809,55.151201,2850000,146.972546
2,25.063302,55.137728,1150000,181.253753
3,25.227295,55.341761,2850000,187.664060
4,25.114275,55.139764,1729200,47.101821


In [4]:
from sklearn.utils import resample

import numpy as np

import math

In [5]:
#Configuracion inicial aleatoria de los clusters:

n=len(Data)

k=4

elementos_clusterizados = []

m = resample(range(0, n), n_samples=math.floor(n/k) , replace=False, random_state=123)

Cluster_0 = Data.loc[m,:]

elementos_clusterizados.append(m)


###################################################################################


if k >= 2:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_1 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_1 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 2 :

    pass

###################################################################################

if k >= 3:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_2 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_2 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 3 :

    pass

###################################################################################

if k >= 4:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_3 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_3 = Data.loc[m ,:]

        elementos_clusterizados.append(m)

elif k < 4 :

    pass

In [6]:
lista_clusters = [None] * 4

lista_clusters[0] = Cluster_0
lista_clusters[1] = Cluster_1
lista_clusters[2] = Cluster_2
lista_clusters[3] = Cluster_3

In [7]:
# Calculo de centroides iniciales

centroide_0 = lista_clusters[0].mean()
centroide_1 = lista_clusters[1].mean()
centroide_2 = lista_clusters[2].mean()
centroide_3 = lista_clusters[3].mean()

# Calculo de distancias Euclideas para la observacion x_1

x_1 = Data.iloc[0,:]

distancia_0 = sum((x_1 - centroide_0)**2)
distancia_1 = sum((x_1 - centroide_1)**2)
distancia_2 = sum((x_1 - centroide_2)**2)
distancia_3 = sum((x_1 - centroide_3)**2)

# Calculo de cluster optimo para x_1

df_distancias = pd.DataFrame({'Distancias' : [distancia_0, distancia_1 , distancia_2 , distancia_3], 'Cluster': [0,1,2,3]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

j_star = df_distancias_sort.iloc[0]['Cluster']

j_star = int(j_star)

In [8]:
j_star

2

In [9]:
i_list = []
i = 1

while i <= n-2 :

    for j in range(0, k) :


        # Si x_i no esta en el cluster j --> pasamos a analizar otro cluster j

        if sum(lista_clusters[j].index == i-1) ==  0 :

            pass


        # Si x_i esta en el cluster j y es el cluster optimo de x_i --> pasamos a x_i+1

        if ( sum(lista_clusters[j].index == i-1) != 0 )  & ( j_star == j ) :

            # Actualizamos i a i+1

            i = i + 1

            i_list.append(i)

            # calculamos las distancias entre x_i+1 y los centroides

            x_i = Data.iloc[ i ,:] # x_i+1

            distancia_0 = sum((x_i - centroide_0)**2)
            distancia_1 = sum((x_i - centroide_1)**2)
            distancia_2 = sum((x_i - centroide_2)**2)
            distancia_3 = sum((x_i - centroide_3)**2)


            # Calculo de cluster optimo para x_1

            df_distancias = pd.DataFrame({'Distancias' : [distancia_0 , distancia_1 , distancia_2, distancia_3], 'Cluster': [0,1,2,3]})

            df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

            j_star = df_distancias_sort.iloc[0]['Cluster']

            j_star = int(j_star)



        # Si x_i esta en el cluster j pero ese no es el cluster optimo de x_i 

        if ( sum(lista_clusters[j].index == i-1) != 0 )  & ( j_star != j ) : 

            ## Actualizamos los clusters:

            #Añadimos x_i al cluster j_star :

            lista_clusters[j_star] = pd.concat( [ lista_clusters[j_star] , lista_clusters[j].loc[i-1,:].to_frame().T ]  )  

            
            # Eliminamos x_1 del cluster j :

            lista_clusters[j] = lista_clusters[j].drop(i-1)   


            # Recalculo de centroides para la nueva configuracion de clusters

            centroide_0 = lista_clusters[0].mean()
            centroide_1 = lista_clusters[1].mean()
            centroide_2 = lista_clusters[2].mean()
            centroide_3 = lista_clusters[3].mean()

            # Recalculo de distancias de x_i respecto los centroides:

            x_i = Data.iloc[i-1,:]

            distancia_0 = sum((x_1 - centroide_0)**2)
            distancia_1 = sum((x_1 - centroide_1)**2)
            distancia_2 = sum((x_1 - centroide_2)**2)
            distancia_3 = sum((x_1 - centroide_3)**2)


In [10]:
# Clusters finales

lista_clusters

[       latitude  longitude       price  size_in_m_2
 1004  25.087251  55.145574   2990000.0   162.208638
 43    25.091487  55.172015   1150000.0   105.259099
 1214  25.078367  55.140410   1650000.0   166.575079
 351   25.111108  55.137591   1699000.0   124.304214
 182   25.064767  55.138814    875000.0   123.282281
 ...         ...        ...         ...          ...
 1871  25.099380  55.141275   1352888.0    69.212735
 1877  25.072573  55.131009   1499000.0   175.029252
 1879  25.203553  55.345554    890000.0    70.606280
 1883  25.072569  55.126527   1970000.0    81.940446
 1885  25.103972  55.149621  31440000.0   607.771426
 
 [517 rows x 4 columns],
        latitude  longitude      price  size_in_m_2
 1041  25.026191  55.156948   350000.0    35.117334
 1282  25.198796  55.271342  2500000.0   168.804751
 1476  25.081955  55.138863   690000.0    66.332742
 1251  25.077150  55.135515  1550000.0   127.648722
 1091  25.206412  55.275199  3950000.0   243.498763
 ...         ...        .

In [11]:
lista_clusters[0].mean()

latitude       2.511612e+01
longitude      5.521077e+01
price          2.051995e+06
size_in_m_2    1.297637e+02
dtype: float64

In [12]:
lista_clusters[1].mean()

latitude       2.511925e+01
longitude      5.521314e+01
price          1.967974e+06
size_in_m_2    1.280963e+02
dtype: float64

In [13]:
lista_clusters[2].mean()

latitude       2.511636e+01
longitude      5.521152e+01
price          2.114565e+06
size_in_m_2    1.346380e+02
dtype: float64

In [14]:
lista_clusters[3].mean()

latitude       2.511513e+01
longitude      5.521413e+01
price          2.179182e+06
size_in_m_2    1.332627e+02
dtype: float64

# k-medoids

In [15]:
#Configuracion inicial aleatoria de los clusters:

n=len(Data)

k=4

elementos_clusterizados = []

m = resample(range(0, n), n_samples=math.floor(n/k) , replace=False, random_state=123)

Cluster_0 = Data.loc[m,:]

elementos_clusterizados.append(m)


###################################################################################


if k >= 2:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_1 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_1 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 2 :

    pass

###################################################################################

if k >= 3:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_2 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_2 = Data.loc[m,:]

        elementos_clusterizados.append(m)

elif k < 3 :

    pass

###################################################################################

if k >= 4:

# Si los elementos que quedan por clusterizar menos el tamaño de los clusters es menor que el propio tamaño de los clusters,
#  se meten todos los elementos que quedan por clusterizar en un mimsmo cluster que será ademas el ultimo.


    if len(np.delete(range(0,n), elementos_clusterizados)) - n/k  < n/k : 

        Cluster_3 = Data.loc[np.delete(range(0,n), elementos_clusterizados),:]

        elementos_clusterizados.append(m)

    else:

        m = resample(np.delete(range(0,n), elementos_clusterizados), n_samples=math.floor(n/k) , replace=False, random_state=123)

        Cluster_3 = Data.loc[m ,:]

        elementos_clusterizados.append(m)

elif k < 4 :

    pass

In [16]:
lista_clusters = [None] * 4

lista_clusters[0] = Cluster_0
lista_clusters[1] = Cluster_1
lista_clusters[2] = Cluster_2
lista_clusters[3] = Cluster_3

In [17]:
distancias_clusters_0 =  np.empty((len(lista_clusters[0]), len(lista_clusters[0])))

distancias_clusters_0[:] = 0

for i in range(0, len(lista_clusters[0])) :

    for r in  range(0, len(lista_clusters[0])) :

            distancias_clusters_0[i,r] = sum( (lista_clusters[0].iloc[i , :] - lista_clusters[0].iloc[r , :])**2 )


In [18]:
# Calculamos la suma de la distancia entre x_i y el resto de observaciones, para cada i=1,..,n

suma_distancias_cluster_0 = []

for i in range(0, distancias_clusters_0.shape[0]):

    suma_distancias_cluster_0.append( distancias_clusters_0[i,:].sum() )

In [19]:
min( suma_distancias_cluster_0 )

4883106009973739.0

In [20]:
np.where(suma_distancias_cluster_0 == min( suma_distancias_cluster_0 ) )

# La observacion medioid del cluster 0 es la que ocupa la fila 409 de la matriz de distancias del cluster, que es la 1002 del cluster 0 (ver mas abajo)

(array([409], dtype=int64),)

In [21]:
suma_distancias_cluster_0[409]

4883106009973739.0

In [22]:
lista_clusters[0].iloc[409,].name

1002

Comprobamos que la observacion 1002 del cluster0 es la de la fila 409 d ela matriz de distancias, y por tanto es el medoid del cluster 0

In [23]:
lista=[]

for r in lista_clusters[0].index :

    lista.append( sum( (lista_clusters[0].loc[1002 , :] - lista_clusters[0].loc[r , :])**2 ))


In [24]:
sum(lista) # coincide con suma_distancias_cluster_0[409]

4883106009973739.0

In [53]:
distancias_clusters_0 =  np.empty((len(lista_clusters[0]), len(lista_clusters[0])))

distancias_clusters_0[:] = 0

for i in range(0, len(lista_clusters[0])) :

    for r in  range(0, len(lista_clusters[0])) :

            distancias_clusters_0[i,r] = sum( (lista_clusters[0].iloc[i , :] - lista_clusters[0].iloc[r , :])**2 )


# Para el cluster 0 : calculamos la suma de la distancia entre x_i y el resto de observaciones, para cada i=1,..,n 

suma_distancias_cluster_0 = []

for i in range(0, distancias_clusters_0.shape[0]):

    suma_distancias_cluster_0.append( distancias_clusters_0[i,:].sum() )


medoids =[]

medoids.append( lista_clusters[0].iloc[ np.where(suma_distancias_cluster_0 == min( suma_distancias_cluster_0 ) )[0] , :].index )


##########################################################################################

distancias_clusters_1 =  np.empty((len(lista_clusters[1]), len(lista_clusters[1])))

distancias_clusters_1[:] = 0

for i in range(0, len(lista_clusters[1])) :

    for r in  range(0, len(lista_clusters[1])) :

            distancias_clusters_1[i,r] = sum( (lista_clusters[1].iloc[i , :] - lista_clusters[1].iloc[r , :])**2 )



# Para el cluster 1 : calculamos la suma de la distancia entre x_i y el resto de observaciones, para cada i=1,..,n 

suma_distancias_cluster_1 = []

for i in range(0, distancias_clusters_1.shape[0]):

    suma_distancias_cluster_1.append( distancias_clusters_1[i,:].sum() )


medoids.append( lista_clusters[1].iloc[ np.where(suma_distancias_cluster_1 == min( suma_distancias_cluster_1 ) )[0] , :].index )

##########################################################################################

distancias_clusters_2 =  np.empty((len(lista_clusters[2]), len(lista_clusters[2])))

distancias_clusters_2[:] = 0

for i in range(0, len(lista_clusters[2])) :

    for r in  range(0, len(lista_clusters[2])) :

            distancias_clusters_2[i,r] = sum( (lista_clusters[2].iloc[i , :] - lista_clusters[2].iloc[r , :])**2 )



# Para el cluster 2 : calculamos la suma de la distancia entre x_i y el resto de observaciones, para cada i=1,..,n 

suma_distancias_cluster_2 = []

for i in range(0, distancias_clusters_2.shape[0]):

    suma_distancias_cluster_2.append( distancias_clusters_2[i,:].sum() )


medoids.append( lista_clusters[2].iloc[ np.where(suma_distancias_cluster_2 == min( suma_distancias_cluster_2 ) )[0] , :].index )

##########################################################################################

distancias_clusters_3 =  np.empty((len(lista_clusters[3]), len(lista_clusters[3])))

distancias_clusters_3[:] = 0

for i in range(0, len(lista_clusters[3])) :

    for r in  range(0, len(lista_clusters[3])) :

            distancias_clusters_3[i,r] = sum( (lista_clusters[3].iloc[i , :] - lista_clusters[3].iloc[r , :])**2 )



# Para el cluster 3 : calculamos la suma de la distancia entre x_i y el resto de observaciones, para cada i=1,..,n 

suma_distancias_cluster_3 = []

for i in range(0, distancias_clusters_3.shape[0]):

    suma_distancias_cluster_3.append( distancias_clusters_3[i,:].sum() )


medoids.append( lista_clusters[3].iloc[ np.where(suma_distancias_cluster_3 == min( suma_distancias_cluster_3 ) )[0] , :].index )


In [54]:
medoids 

# Ha tardado 7.38 mins en calcular los medoids, tarda demasiado. 
#Si tarda esto cada vez que se tengan que actualizar los medoids, esta implementacion del algoritmo no es utilizable en la practica

[Int64Index([1002], dtype='int64'),
 Int64Index([67], dtype='int64'),
 Int64Index([1775], dtype='int64'),
 Int64Index([198], dtype='int64')]

In [ ]:
# Calculo de centroides iniciales

centroide_0 = lista_clusters[0].mean()
centroide_1 = lista_clusters[1].mean()
centroide_2 = lista_clusters[2].mean()
centroide_3 = lista_clusters[3].mean()


In [ ]:

# Calculo de distancias Euclideas para la observacion x_1

x_1 = Data.iloc[0,:]

distancia_0 = sum((x_1 - centroide_0)**2)
distancia_1 = sum((x_1 - centroide_1)**2)
distancia_2 = sum((x_1 - centroide_2)**2)
distancia_3 = sum((x_1 - centroide_3)**2)

# Calculo de cluster optimo para x_1

df_distancias = pd.DataFrame({'Distancias' : [distancia_0, distancia_1 , distancia_2 , distancia_3], 'Cluster': [0,1,2,3]})

df_distancias_sort = df_distancias.sort_values(by='Distancias', ascending=False)

j_star = df_distancias_sort.iloc[0]['Cluster']

j_star = int(j_star)